In [1]:
pip install azure-cognitiveservices-speech


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import azure.cognitiveservices.speech as speechsdk

# Replace with your own subscription key and service region (e.g., "westus").
subscription_key = "8d0afea35c1542b4b685e81608ad4e0b"
service_region = "centralindia"
audio_filename = "India top 10 bank.wav"  # Replace with the path to your audio file.

def transcribe_audio_to_text(subscription_key, service_region, audio_filename):
    # Create an instance of a speech config with specified subscription key and service region.
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)

    # Specify the audio file to be transcribed.
    audio_input = speechsdk.AudioConfig(filename=audio_filename)

    # Create a recognizer with the given settings.
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

    # Transcribe the audio and handle the results.
    def result_callback(event_type, evt):
        if event_type == speechsdk.EventSignalType.SPEECH_RECOGNIZED:
            print(f"RECOGNIZED: {evt.result.text}")
        elif event_type == speechsdk.EventSignalType.SPEECH_END_DETECTED:
            print(f"Speech end detected: {evt}")
        elif event_type == speechsdk.EventSignalType.RECOGNIZED:
            print(f"Recognized: {evt.result.text}")

    # Connect callbacks to the recognizer events.
    speech_recognizer.recognized.connect(lambda evt: result_callback(speechsdk.EventSignalType.RECOGNIZED, evt))
    speech_recognizer.recognizing.connect(lambda evt: result_callback(speechsdk.EventSignalType.SPEECH_RECOGNIZED, evt))
    speech_recognizer.session_stopped.connect(lambda evt: print("Session stopped"))
    speech_recognizer.canceled.connect(lambda evt: print(f"CANCELED: {evt.reason}"))

    # Start transcription.
    print("Starting transcription...")
    result = speech_recognizer.start_continuous_recognition()
    result.wait_for_final_result()

    # Stop recognition after all events are processed.
    print("Stopping transcription...")
    speech_recognizer.stop_continuous_recognition()

# Run the transcription function.
transcribe_audio_to_text(subscription_key, service_region, audio_filename)


Starting transcription...


AttributeError: 'NoneType' object has no attribute 'wait_for_final_result'

Session stopped


In [1]:
import azure.cognitiveservices.speech as speechsdk

def transcribe_with_diarization(subscription_key, service_region, audio_filename):
    # Create a speech configuration
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)

    # Create an audio configuration
    audio_config = speechsdk.audio.AudioConfig(filename=audio_filename)

    # Enable speaker diarization
    speech_config.set_property(speechsdk.PropertyId.SpeechServiceConnection_ContinuousLanguageIdPriority, "High")

    # Create a speech recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    # Define the callback function to handle recognized results
    def handle_final_result(evt):
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            print(f"Recognized: {evt.result.text}")
            json_result = evt.result.json
            print(f"JSON result: {json_result}")
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print(f"No speech could be recognized: {evt.result.no_match_details}")

    # Define the callback function to handle cancellations
    def handle_canceled(evt):
        print(f"CANCELED: Reason={evt.reason}")
        if evt.reason == speechsdk.CancellationReason.Error:
            print(f"CANCELED: ErrorDetails={evt.error_details}")

    done = False

    # Define the callback function to stop recognition
    def stop_cb(evt):
        nonlocal done
        done = True

    # Connect the event handlers
    speech_recognizer.recognized.connect(handle_final_result)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(handle_canceled)

    # Start continuous recognition
    speech_recognizer.start_continuous_recognition()
    print("Recognition started...")

    while not done:
        pass

    # Stop recognition
    speech_recognizer.stop_continuous_recognition()
    print("Recognition stopped.")

def main(subscription_key, service_region, audio_filename):
    transcribe_with_diarization(subscription_key, service_region, audio_filename)

if __name__ == "__main__":
    subscription_key = '8d0afea35c1542b4b685e81608ad4e0b'
    service_region = 'centralindia'
    audio_filename = 'pcm_format.wav'

    main(subscription_key, service_region, audio_filename)


AttributeError: SpeechServiceConnection_ContinuousLanguageIdPriority

In [6]:
import azure.cognitiveservices.speech as speechsdk

def transcribe_with_diarization(subscription_key, service_region, audio_filename):
    # Create a speech configuration
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)

    # Create an audio configuration
    audio_config = speechsdk.audio.AudioConfig(filename=audio_filename)

    # Enable speaker diarization
    speech_config.set_property_by_name("SpeechServiceResponse_SpeakerDiarization", "true")

    # Create a speech recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False

    # Define the callback function to handle recognized results
    def handle_final_result(evt):
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            print(f"Recognized: {evt.result.text}")
            json_result = evt.result.json
            print(f"JSON result: {json_result}")
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print(f"No speech could be recognized: {evt.result.no_match_details}")

    # Define the callback function to handle cancellations
    def handle_canceled(evt):
        print(f"CANCELED: Reason={evt.reason}")
        if evt.reason == speechsdk.CancellationReason.Error:
            print(f"CANCELED: ErrorDetails={evt.error_details}")
        nonlocal done
        done = True

    # Define the callback function to stop recognition
    def stop_cb(evt):
        nonlocal done
        done = True

    # Connect the event handlers
    speech_recognizer.recognized.connect(handle_final_result)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(handle_canceled)

    # Start continuous recognition
    speech_recognizer.start_continuous_recognition()
    print("Recognition started...")

    while not done:
        pass

    # Stop recognition
    speech_recognizer.stop_continuous_recognition()
    print("Recognition stopped.")

def main(subscription_key, service_region, audio_filename):
    transcribe_with_diarization(subscription_key, service_region, audio_filename)

if __name__ == "__main__":
    subscription_key = "d4b356c12208408bb9da697544677eb8"
    service_region = "southeastasia"
    audio_filename = "hindi_audio.wav"
    
    main(subscription_key, service_region, audio_filename)


Recognition started...
Recognized: Career clips Hindi OK OK OK subscribe karo Bella icon but click karo or seek tejao life mehdoor chalta.
JSON result: {"Id":"841eead3597b4de9a494fa8f16fd911d","RecognitionStatus":"Success","DisplayText":"Career clips Hindi OK OK OK subscribe karo Bella icon but click karo or seek tejao life mehdoor chalta.","Offset":2300000,"Duration":94800000,"Channel":0}
Recognized: Podcast future because log you know app swipe karayo appo time pass eh actually rookie was he kani right? Was it time pass karai ORU usmaye entertaining cheese eh just jisma AAP kulakthaiya both thagyavathu relax, relax sunaiya mujhe to AAP NI lip sync cafe dance kisi ke dekhliye kitna Dey dekhlu.
JSON result: {"Id":"8b30a406911e40a2ac8edf4a2c5532c6","RecognitionStatus":"Success","DisplayText":"Podcast future because log you know app swipe karayo appo time pass eh actually rookie was he kani right? Was it time pass karai ORU usmaye entertaining cheese eh just jisma AAP kulakthaiya both th

In [7]:
import azure.cognitiveservices.speech as speechsdk

def transcribe_with_diarization(subscription_key, service_region, audio_filename):
    # Create a speech configuration
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)

    # Create an audio configuration
    audio_config = speechsdk.audio.AudioConfig(filename=audio_filename)

    # Enable speaker diarization
    speech_config.set_property_by_name("SpeechServiceResponse_SpeakerDiarization", "true")

    # Create a speech recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False

    # Define the callback function to handle recognized results
    def handle_final_result(evt):
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            print(f"Recognized: {evt.result.text}")
            json_result = evt.result.json
            print(f"JSON result: {json_result}")
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print(f"No speech could be recognized: {evt.result.no_match_details}")

    # Define the callback function to handle cancellations
    def handle_canceled(evt):
        print(f"CANCELED: Reason={evt.reason}")
        if evt.reason == speechsdk.CancellationReason.Error:
            print(f"CANCELED: ErrorDetails={evt.error_details}")
        nonlocal done
        done = True

    # Define the callback function to stop recognition
    def stop_cb(evt):
        nonlocal done
        done = True

    # Connect the event handlers
    speech_recognizer.recognized.connect(handle_final_result)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(handle_canceled)

    # Start continuous recognition
    speech_recognizer.start_continuous_recognition()
    print("Recognition started...")

    # Wait for the completion of the recognition
    while not done:
        pass

    # Stop recognition
    speech_recognizer.stop_continuous_recognition()
    print("Recognition stopped.")

def main(subscription_key, service_region, audio_filename):
    transcribe_with_diarization(subscription_key, service_region, audio_filename)

if __name__ == "__main__":
    subscription_key = "d4b356c12208408bb9da697544677eb8"
    service_region = "southeastasia"
    audio_filename = "hindi_audio.wav"
    
    
    main(subscription_key, service_region, audio_filename)


Recognition started...
Recognized: Career clips Hindi OK OK OK subscribe karo Bella icon but click karo or seek tejao life mehdoor chalta.
JSON result: {"Id":"3a912b1f9ad343a5b8e31e75ee0c6343","RecognitionStatus":"Success","DisplayText":"Career clips Hindi OK OK OK subscribe karo Bella icon but click karo or seek tejao life mehdoor chalta.","Offset":2300000,"Duration":94800000,"Channel":0}
Recognized: Podcast future because log you know app swipe karayo appo time pass eh actually rookie was he kani right? Was it time pass karai ORU usmaye entertaining cheese eh just jisma AAP kulakthaiya both thagyavathu relax, relax sunaiya mujhe to AAP NI lip sync cafe dance kisi ke dekhliye kitna Dey dekhlu.
JSON result: {"Id":"c6864e3733ea49a5aef15f2c9731cd97","RecognitionStatus":"Success","DisplayText":"Podcast future because log you know app swipe karayo appo time pass eh actually rookie was he kani right? Was it time pass karai ORU usmaye entertaining cheese eh just jisma AAP kulakthaiya both th

In [9]:
import azure.cognitiveservices.speech as speechsdk

def transcribe_with_diarization(subscription_key, service_region, audio_filename):
    # Create a speech configuration
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)

    # Create an audio configuration
    audio_config = speechsdk.audio.AudioConfig(filename=audio_filename)

    # Enable speaker diarization
    speech_config.set_property_by_name("SpeechServiceResponse_SpeakerDiarization", "true")

    # Create a speech recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False

    # Define the callback function to handle recognized results
    def handle_final_result(evt):
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            print(f"Recognized: {evt.result.text}")
            json_result = evt.result.json
            print(f"JSON result: {json_result}")
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print(f"No speech could be recognized: {evt.result.no_match_details}")

    # Define the callback function to handle cancellations
    def handle_canceled(evt):
        print(f"CANCELED: Reason={evt.reason}")
        if evt.reason == speechsdk.CancellationReason.Error:
            print(f"CANCELED: ErrorDetails={evt.error_details}")
        nonlocal done
        done = True

    # Define the callback function to stop recognition
    def stop_cb(evt):
        nonlocal done
        done = True

    # Connect the event handlers
    speech_recognizer.recognized.connect(handle_final_result)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(handle_canceled)

    # Start continuous recognition
    speech_recognizer.start_continuous_recognition()
    print("Recognition started...")

    # Wait for the completion of the recognition
    while not done:
        pass

    # Stop recognition
    speech_recognizer.stop_continuous_recognition()
    print("Recognition stopped.")

def main(subscription_key, service_region, audio_filename):
    transcribe_with_diarization(subscription_key, service_region, audio_filename)

if __name__ == "__main__":
    subscription_key = "d4b356c12208408bb9da697544677eb8"
    service_region = "southeastasia"
    audio_filename = "hindi_audio.wav"
    
    
    main(subscription_key, service_region, audio_filename)


Recognition started...
Recognized: Career clips Hindi OK OK OK subscribe karo Bella icon but click karo or seek tejao life mehdoor chalta.
JSON result: {"Id":"ea482cace399461ea52125da1c4cc26d","RecognitionStatus":"Success","DisplayText":"Career clips Hindi OK OK OK subscribe karo Bella icon but click karo or seek tejao life mehdoor chalta.","Offset":2300000,"Duration":94800000,"Channel":0}
Recognized: Podcast future because log you know app swipe karayo appo time pass eh actually rookie was he kani right? Was it time pass karai ORU usmaye entertaining cheese eh just jisma AAP kulakthaiya both thagyavathu relax, relax sunaiya mujhe to AAP NI lip sync cafe dance kisi ke dekhliye kitna Dey dekhlu.
JSON result: {"Id":"72e39f9c1a2d4b5f8b1eba41e3040ec2","RecognitionStatus":"Success","DisplayText":"Podcast future because log you know app swipe karayo appo time pass eh actually rookie was he kani right? Was it time pass karai ORU usmaye entertaining cheese eh just jisma AAP kulakthaiya both th

In [14]:
import azure.cognitiveservices.speech as speechsdk

def transcribe_with_diarization(subscription_key, service_region, audio_filename):
    # Create a speech configuration
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)

    # Create an audio configuration
    audio_config = speechsdk.audio.AudioConfig(filename=audio_filename)

    # Enable speaker diarization
    speech_config.set_property_by_name("SpeechServiceResponse_SpeakerDiarization", "true")

    # Create a speech recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False

    # Define the callback function to handle recognized results
    def handle_final_result(evt):
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            print(f"Recognized: {evt.result.text}")
            json_result = evt.result.json
            print(f"JSON result: {json_result}")
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print(f"No speech could be recognized: {evt.result.no_match_details}")

    # Define the callback function to handle cancellations
    def handle_canceled(evt):
        print(f"CANCELED: Reason={evt.reason}")
        if evt.reason == speechsdk.CancellationReason.Error:
            print(f"CANCELED: ErrorDetails={evt.error_details}")
        nonlocal done
        done = True

    # Define the callback function to stop recognition
    def stop_cb(evt):
        nonlocal done
        done = True

    # Connect the event handlers
    speech_recognizer.recognized.connect(handle_final_result)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(handle_canceled)

    # Start continuous recognition
    speech_recognizer.start_continuous_recognition()
    print("Recognition started...")

    # Wait for the completion of the recognition
    while not done:
        pass

    # Stop recognition
    speech_recognizer.stop_continuous_recognition()
    print("Recognition stopped.")

def main(subscription_key, service_region, audio_filename):
    transcribe_with_diarization(subscription_key, service_region, audio_filename)

if __name__ == "__main__":
    subscription_key = "d4b356c12208408bb9da697544677eb8"
    service_region = "southeastasia"
    audio_filename = "hindi_audio.wav"
    
    
    main(subscription_key, service_region, audio_filename)


Recognition started...
Recognized: Career clips Hindi OK OK OK subscribe karo Bella icon but click karo or seek tejao life mehdoor chalta.
JSON result: {"Id":"b12b4351bcda46348f884dc74f34b415","RecognitionStatus":"Success","DisplayText":"Career clips Hindi OK OK OK subscribe karo Bella icon but click karo or seek tejao life mehdoor chalta.","Offset":2300000,"Duration":94800000,"Channel":0}
Recognized: Podcast future because log you know app swipe karayo appo time pass eh actually rookie was he kani right? Was it time pass karai ORU usmaye entertaining cheese eh just jisma AAP kulakthaiya both thagyavathu relax, relax sunaiya mujhe to AAP NI lip sync cafe dance kisi ke dekhliye kitna Dey dekhlu.
JSON result: {"Id":"9543f4dd15e043db99e87834fe6976fb","RecognitionStatus":"Success","DisplayText":"Podcast future because log you know app swipe karayo appo time pass eh actually rookie was he kani right? Was it time pass karai ORU usmaye entertaining cheese eh just jisma AAP kulakthaiya both th

In [8]:
import azure.cognitiveservices.speech as speechsdk
import json

def transcribe_with_diarization(subscription_key, service_region, audio_filename):
    # Create a speech configuration
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)

    # Enable speaker diarization
    speech_config.set_property_by_name("SpeechServiceResponse_SpeakerDiarization", "true")

    # Create an audio configuration
    audio_config = speechsdk.audio.AudioConfig(filename=audio_filename)

    # Create a speech recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False
    results = []

    # Define the callback function to handle recognized results
    def handle_final_result(evt):
        nonlocal results
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            speaker_id = evt.result.properties.get(speechsdk.PropertyId.SpeechServiceResponse_JsonResult_SpeakerIdentifier)
            start_time = evt.result.offset_in_ticks / 10**7  # convert ticks to seconds
            end_time = (evt.result.offset_in_ticks + evt.result.duration_in_ticks) / 10**7  # convert ticks to seconds
            confidence = evt.result.properties.get(speechsdk.PropertyId.SpeechServiceResponse_JsonResult_Confidence)
            result = {
                "speaker": speaker_id,
                "sentence": evt.result.text,
                "start_time": start_time,
                "end_time": end_time,
                "confidence": confidence
            }
            results.append(result)

    # Define the callback function to handle cancellations
    def handle_canceled(evt):
        print(f"CANCELED: Reason={evt.reason}")
        if evt.reason == speechsdk.CancellationReason.Error:
            print(f"CANCELED: ErrorDetails={evt.error_details}")
        nonlocal done
        done = True

    # Define the callback function to stop recognition
    def stop_cb(evt):
        nonlocal done
        done = True

    # Connect the event handlers
    speech_recognizer.recognized.connect(handle_final_result)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(handle_canceled)

    # Start continuous recognition
    speech_recognizer.start_continuous_recognition()

    print("Recognition started...")

    while not done:
        pass

    # Stop recognition
    speech_recognizer.stop_continuous_recognition()
    print("Recognition stopped.")

    return results

def main(subscription_key, service_region, audio_filename):
    results = transcribe_with_diarization(subscription_key, service_region, audio_filename)
    print(json.dumps(results, indent=2))  # Print results in JSON format

if __name__ == "__main__":
    subscription_key = "d4b356c12208408bb9da697544677eb8"
    service_region = "southeastasia"
    audio_filename = "hindi_audio.wav"
    

    main(subscription_key, service_region, audio_filename)


Recognition started...
Recognition stopped.
[]


In [11]:
import azure.cognitiveservices.speech as speechsdk
import json

def transcribe_with_diarization(subscription_key, service_region, audio_filename):
    # Create a speech configuration
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)

    # Enable speaker diarization
    speech_config.set_property_by_name("SpeechServiceResponse_SpeakerDiarization", "true")

    # Create an audio configuration
    audio_config = speechsdk.audio.AudioConfig(filename=audio_filename)

    # Create a speech recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False
    results = []

    # Define the callback function to handle recognized results
    def handle_final_result(evt):
        nonlocal results
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            speaker_id = evt.result.properties.get(speechsdk.PropertyId.SpeechServiceResponse_JsonResult_SpeakerIdentifier)
            start_time = evt.result.offset_in_ticks / 10**7  # convert ticks to seconds
            end_time = (evt.result.offset_in_ticks + evt.result.duration_in_ticks) / 10**7  # convert ticks to seconds
            confidence = evt.result.properties.get(speechsdk.PropertyId.SpeechServiceResponse_JsonResult_Confidence)
            result = {
                "speaker": speaker_id,
                "sentence": evt.result.text,
                "start_time": start_time,
                "end_time": end_time,
                "confidence": confidence
            }
            results.append(result)

    # Define the callback function to handle cancellations
    def handle_canceled(evt):
        print(f"CANCELED: Reason={evt.reason}")
        if evt.reason == speechsdk.CancellationReason.Error:
            print(f"CANCELED: ErrorDetails={evt.error_details}")
        nonlocal done
        done = True

    # Define the callback function to stop recognition
    def stop_cb(evt):
        nonlocal done
        done = True

    # Connect the event handlers
    speech_recognizer.recognized.connect(handle_final_result)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(handle_canceled)

    # Start continuous recognition
    speech_recognizer.start_continuous_recognition()

    print("Recognition started...")

    while not done:
        pass

    # Stop recognition
    speech_recognizer.stop_continuous_recognition()
    print("Recognition stopped.")

    return results

def main(subscription_key, service_region, audio_filename):
    results = transcribe_with_diarization(subscription_key, service_region, audio_filename)
    print(json.dumps(results, indent=2))  # Print results in JSON format

if __name__ == "__main__":
    subscription_key = "d4b356c12208408bb9da697544677eb8"
    service_region = "southeastasia"
    audio_filename = r"C:\Users\HP\Downloads\VS_CODE\pcm _format.wav"
    

    main(subscription_key, service_region, audio_filename)


Recognition started...
Recognition stopped.
[]


In [5]:
import azure.cognitiveservices.speech as speechsdk
import time

def transcribe_with_diarization(subscription_key, service_region, audio_filename):
    # Create a speech configuration
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)

    # Create an audio configuration
    audio_config = speechsdk.audio.AudioConfig(filename=audio_filename)

    # Enable speaker diarization
    speech_config.set_property_by_name("SpeechServiceResponse_SpeakerDiarization", "true")

    # Create a speech recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False

    # Define the callback function to handle recognized results
    def handle_final_result(evt):
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            print(f"Recognized: {evt.result.text}")
            json_result = evt.result.json
            print(f"JSON result: {json_result}")
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print(f"No speech could be recognized: {evt.result.no_match_details}")

    # Define the callback function to handle cancellations
    def handle_canceled(evt):
        print(f"CANCELED: Reason={evt.reason}")
        if evt.reason == speechsdk.CancellationReason.Error:
            print(f"CANCELED: ErrorDetails={evt.error_details}")
        nonlocal done
        done = True

    # Define the callback function to stop recognition
    def stop_cb(evt):
        nonlocal done
        done = True

    # Connect the event handlers
    speech_recognizer.recognized.connect(handle_final_result)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(handle_canceled)

    # Start continuous recognition
    speech_recognizer.start_continuous_recognition()
    print("Recognition started...")

    # Wait for the completion of the recognition
    while not done:
        time.sleep(0.1)  # Adding sleep to reduce CPU usage

    # Stop recognition
    speech_recognizer.stop_continuous_recognition()
    print("Recognition stopped.")

def main(subscription_key, service_region, audio_filename):
    transcribe_with_diarization(subscription_key, service_region, audio_filename)

if __name__ == "__main__":
    subscription_key = "8d0afea35c1542b4b685e81608ad4e0b"
    service_region = "centralindia"
    audio_filename = r"C:\Users\HP\Downloads\VS_CODE\pcm _format.wav"
    
    main(subscription_key, service_region, audio_filename)


Recognition started...
Recognition stopped.


In [12]:
import azure.cognitiveservices.speech as speechsdk
import json

def transcribe_with_diarization(subscription_key, service_region, audio_filename):
    # Create a speech configuration
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)

    # Enable speaker diarization
    speech_config.set_property(speechsdk.PropertyId.SpeechServiceConnection_ContinuousLanguageIdPriority, "High")

    # Create an audio configuration
    audio_config = speechsdk.audio.AudioConfig(filename=audio_filename)

    # Create a speech recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False
    results = []

    # Define the callback function to handle recognized results
    def handle_final_result(evt):
        nonlocal results
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            speaker_id = evt.result.properties.get(speechsdk.PropertyId.SpeechServiceResponse_JsonResult_SpeakerIdentifier)
            start_time = evt.result.offset_in_ticks / 10**7  # convert ticks to seconds
            end_time = (evt.result.offset_in_ticks + evt.result.duration_in_ticks) / 10**7  # convert ticks to seconds
            confidence = evt.result.properties.get(speechsdk.PropertyId.SpeechServiceResponse_JsonResult_Confidence)
            result = {
                "speaker": speaker_id,
                "sentence": evt.result.text,
                "start_time": start_time,
                "end_time": end_time,
                "confidence": confidence
            }
            results.append(result)

    # Define the callback function to handle cancellations
    def handle_canceled(evt):
        print(f"CANCELED: Reason={evt.reason}")
        if evt.reason == speechsdk.CancellationReason.Error:
            print(f"CANCELED: ErrorDetails={evt.error_details}")
        nonlocal done
        done = True

    # Define the callback function to stop recognition
    def stop_cb(evt):
        nonlocal done
        done = True

    # Connect the event handlers
    speech_recognizer.recognized.connect(handle_final_result)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(handle_canceled)

    # Start continuous recognition
    speech_recognizer.start_continuous_recognition()

    print("Recognition started...")

    while not done:
        pass

    # Stop recognition
    speech_recognizer.stop_continuous_recognition()
    print("Recognition stopped.")

    return results

def main(subscription_key, service_region, audio_filename):
    try:
        results = transcribe_with_diarization(subscription_key, service_region, audio_filename)
        if results:
            print(json.dumps(results, indent=2))  # Print results in JSON format
        else:
            print("No speech recognized.")
    except Exception as ex:
        print(f"Exception: {ex}")

if __name__ == "__main__":
    subscription_key = "8d0afea35c1542b4b685e81608ad4e0b"
    service_region = "centralindia"
    audio_filename = r"C:\Users\HP\Downloads\VS_CODE\pcm _format.wav"
    

    main(subscription_key, service_region, audio_filename)


Exception: SpeechServiceConnection_ContinuousLanguageIdPriority


In [13]:
import azure.cognitiveservices.speech as speechsdk
import json

def transcribe_with_diarization(subscription_key, service_region, audio_filename):
    # Create a speech configuration
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)

    # Enable speaker diarization using property name
    speech_config.set_property_by_name("ConversationTranscriptionEnableSpeakerDiarization", "true")

    # Create an audio configuration
    audio_config = speechsdk.audio.AudioConfig(filename=audio_filename)

    # Create a speech recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False
    results = []

    # Define the callback function to handle recognized results
    def handle_final_result(evt):
        nonlocal results
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            speaker_id = evt.result.properties.get("SpeakerId")
            start_time = evt.result.offset_in_ticks / 10**7  # convert ticks to seconds
            end_time = (evt.result.offset_in_ticks + evt.result.duration_in_ticks) / 10**7  # convert ticks to seconds
            confidence = evt.result.properties.get("SpeechServiceResponse_JsonResult_Confidence")
            result = {
                "speaker": speaker_id,
                "sentence": evt.result.text,
                "start_time": start_time,
                "end_time": end_time,
                "confidence": confidence
            }
            results.append(result)

    # Define the callback function to handle cancellations
    def handle_canceled(evt):
        print(f"CANCELED: Reason={evt.reason}")
        if evt.reason == speechsdk.CancellationReason.Error:
            print(f"CANCELED: ErrorDetails={evt.error_details}")
        nonlocal done
        done = True

    # Define the callback function to stop recognition
    def stop_cb(evt):
        nonlocal done
        done = True

    # Connect the event handlers
    speech_recognizer.recognized.connect(handle_final_result)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(handle_canceled)

    # Start continuous recognition
    speech_recognizer.start_continuous_recognition()

    print("Recognition started...")

    while not done:
        pass

    # Stop recognition
    speech_recognizer.stop_continuous_recognition()
    print("Recognition stopped.")

    return results

def main(subscription_key, service_region, audio_filename):
    try:
        results = transcribe_with_diarization(subscription_key, service_region, audio_filename)
        if results:
            print(json.dumps(results, indent=2))  # Print results in JSON format
        else:
            print("No speech recognized.")
    except Exception as ex:
        print(f"Exception: {ex}")

if __name__ == "__main__":
    subscription_key = "8d0afea35c1542b4b685e81608ad4e0b"
    service_region = "centralindia"
    audio_filename = r"C:\Users\HP\Downloads\VS_CODE\pcm _format.wav"
    
   
    main(subscription_key, service_region, audio_filename)


Recognition started...
Recognition stopped.
No speech recognized.


In [3]:
import azure.cognitiveservices.speech as speechsdk

# Replace with your own subscription key and service region (e.g., "westus").
subscription_key = "8d0afea35c1542b4b685e81608ad4e0b"
service_region = "centralindia"
audio_filename = "India top 10 bank.wav"  # Replace with the path to your audio file.

def transcribe_audio_to_text(subscription_key, service_region, audio_filename):
    # Create an instance of a speech config with specified subscription key and service region.
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)

    # Specify the audio file to be transcribed.
    audio_input = speechsdk.AudioConfig(filename=audio_filename)

    # Create a recognizer with the given settings.
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

    # Callback for recognized event
    def recognized(evt):
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            print(f"RECOGNIZED: {evt.result.text}")
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print("No speech could be recognized")

    # Callback for session stopped event
    def session_stopped(evt):
        print("Session stopped")
        done.set()

    # Callback for canceled event
    def canceled(evt):
        print(f"CANCELED: {evt.reason}")
        done.set()

    # Connect callbacks to the recognizer events.
    speech_recognizer.recognized.connect(recognized)
    speech_recognizer.session_stopped.connect(session_stopped)
    speech_recognizer.canceled.connect(canceled)

    # Start transcription.
    print("Starting transcription...")
    speech_recognizer.start_continuous_recognition()
    
    # Wait until the transcription is done.
    import threading
    done = threading.Event()
    done.wait()

    # Stop recognition.
    print("Stopping transcription...")
    speech_recognizer.stop_continuous_recognition()

# Run the transcription function.
transcribe_audio_to_text(subscription_key, service_region, audio_filename)


Starting transcription...
RECOGNIZED: Hello friends.
RECOGNIZED: 
RECOGNIZED: Business.
RECOGNIZED: Foreign bencoco market video coffee interesting hone Wali hai #10 hamari yes bank yes bank Ek Bharti bank.
RECOGNIZED: Kapoor Raso Kapuri AKA Bharat Ki Das subsidi Benko Kaisai Iske Karimi management retail banki Hai Bank.
RECOGNIZED: Kul ATM Ki sanke Bara sobi SE or purideshmis benki Gara subi sakhi mojo hai oris banka total market.
RECOGNIZED: #9.
RECOGNIZED: EC list number no per head Bank of India Ek bhartiya bank hai jiska headquarter Mumbai Mesti is bank office Botswana, New Zealand, France, Singapore, Hong Kong, Japan, Kenya, America, Britain, Jungada, Dakshinoprika, Vietnam or Suzanne.
RECOGNIZED: Today's Bank.
RECOGNIZED: A total market cap key to bank a total market KET bhartiya rupa hai.
RECOGNIZED: #8 Punjab National Bank, East Bank, Total Art Croatia, Jada Customer Mojo, Punjab National Bank, Dubai or Hong Kong.
RECOGNIZED: Headquarter Mumbai, Punjab National Bank.
RECOGNIZE

In [4]:
import azure.cognitiveservices.speech as speechsdk

# Replace with your own subscription key and service region (e.g., "westus").
subscription_key = "8d0afea35c1542b4b685e81608ad4e0b"
service_region = "centralindia"
audio_filename = "India top 10 bank.wav"  # Replace with the path to your audio file.

def transcribe_audio_to_text(subscription_key, service_region, audio_filename):
    # Create an instance of a speech config with specified subscription key and service region.
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)

    # Specify the audio file to be transcribed.
    audio_input = speechsdk.AudioConfig(filename=audio_filename)

    # Create a recognizer with the given settings.
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

    full_transcription = []

    # Callback for recognized event
    def recognized(evt):
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            full_transcription.append(evt.result.text)
            print(f"RECOGNIZED: {evt.result.text}")
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print("No speech could be recognized")

    # Callback for canceled event
    def canceled(evt):
        print(f"CANCELED: {evt.reason}")
        done.set()

    # Connect callbacks to the recognizer events.
    speech_recognizer.recognized.connect(recognized)
    speech_recognizer.canceled.connect(canceled)

    # Start transcription in chunks.
    print("Starting transcription...")
    done = False
    while not done:
        result_future = speech_recognizer.recognize_once_async()
        result = result_future.get()
        if result.reason == speechsdk.ResultReason.Canceled:
            done = True

    # Join all transcriptions
    complete_transcription = ' '.join(full_transcription)
    print("Final transcription:", complete_transcription)
    return complete_transcription

# Run the transcription function.
transcribe_audio_to_text(subscription_key, service_region, audio_filename)


Starting transcription...
RECOGNIZED: Hello friends.
RECOGNIZED: Havradhi saje hai duniya Ki nazar Bani hue.
RECOGNIZED: Varmar desk dunia buti gorshe dekri hai koke mardeski vikas ke sasa Tamara desi Amir hota Zara hai kismi Bharti benko kabhi mukhi rohai sudesh ke logo ko loan de hai business.
No speech could be recognized
RECOGNIZED: Or in benco market Cape coding rank hai video saath koki video coffee interesting hone Wali hai #10 hamari yes bank yes bank Ek bhartiya benke or iski sapna dojar charmiti churana Kapoor or rasoi giti.
No speech could be recognized
RECOGNIZED: Management retail banking Hai Bank.
RECOGNIZED: Attara azar kamariye ATM Ki sanke Bara sobi say or Puri deshmukh benki Gara subi Sakai mojai or is benka total market?
RECOGNIZED: #9.
RECOGNIZED: EC list number now perhab Bank of India, Bhartiya Bank Hai Jiska headquarter Mumbai is Bank office, Botswana, New Zealand, France, Singapore, Hong Kong, Japan, Kenya, Suntrasi, America, Britain, Jungada, Dakshinoprika, Vie

"Hello friends. Havradhi saje hai duniya Ki nazar Bani hue. Varmar desk dunia buti gorshe dekri hai koke mardeski vikas ke sasa Tamara desi Amir hota Zara hai kismi Bharti benko kabhi mukhi rohai sudesh ke logo ko loan de hai business. Or in benco market Cape coding rank hai video saath koki video coffee interesting hone Wali hai #10 hamari yes bank yes bank Ek bhartiya benke or iski sapna dojar charmiti churana Kapoor or rasoi giti. Management retail banking Hai Bank. Attara azar kamariye ATM Ki sanke Bara sobi say or Puri deshmukh benki Gara subi Sakai mojai or is benka total market? #9. EC list number now perhab Bank of India, Bhartiya Bank Hai Jiska headquarter Mumbai is Bank office, Botswana, New Zealand, France, Singapore, Hong Kong, Japan, Kenya, Suntrasi, America, Britain, Jungada, Dakshinoprika, Vietnam or Susania Jessica Chapman. Customer. Chris Banky Total Market, Capki to Bank Total Market. #8. Punjab National Bank, East Bank, Total Art Croatia, Jada Customer Mojo, Punjab N

In [5]:
import azure.cognitiveservices.speech as speechsdk

# Replace with your own subscription key and service region (e.g., "westus").
subscription_key = "8d0afea35c1542b4b685e81608ad4e0b"
service_region = "centralindia"
audio_filename = "hindi_audio.wav"  # Replace with the path to your audio file.

def transcribe_audio_with_diarization(subscription_key, service_region, audio_filename):
    # Create an instance of a speech config with specified subscription key and service region.
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)

    # Enable speaker diarization
    speech_config.request_word_level_timestamps()
    speech_config.enable_audio_logging()
    speech_config.set_property(speechsdk.PropertyId.SpeechServiceResponse_ProfanityOption, "masked")

    # Specify the audio file to be transcribed.
    audio_input = speechsdk.AudioConfig(filename=audio_filename)

    # Create a recognizer with the given settings.
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

    full_transcription = []

    def recognized(evt):
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            full_transcription.append(evt.result.text)
            print(f"RECOGNIZED: Speaker {evt.result.speaker_id}: {evt.result.text}")
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print("No speech could be recognized")

    def canceled(evt):
        print(f"CANCELED: {evt.reason}")
        done.set()

    def session_stopped(evt):
        print("Session stopped")
        done.set()

    # Connect callbacks to the recognizer events.
    speech_recognizer.recognized.connect(recognized)
    speech_recognizer.canceled.connect(canceled)
    speech_recognizer.session_stopped.connect(session_stopped)

    # Start transcription.
    print("Starting transcription...")
    done = False
    while not done:
        result_future = speech_recognizer.recognize_once_async()
        result = result_future.get()
        if result.reason == speechsdk.ResultReason.Canceled:
            done = True

    # Join all transcriptions
    complete_transcription = ' '.join(full_transcription)
    print("Final transcription:", complete_transcription)
    return complete_transcription

# Run the transcription function.
transcribe_audio_with_diarization(subscription_key, service_region, audio_filename)


Starting transcription...
Session stopped
Session stopped
Session stopped
Session stopped
No speech could be recognized
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
No speech could be recognized
Session stopped
No speech could be recognized
Session stopped
No speech could be recognized
Session stopped
No speech could be recognized
Session stopped
No speech could be recognized
Session stopped
Session stopped
Final transcription: Career Maduro Jana isley apne select Kia TRS clips Hindi OK OK OK subscribe karo Bella icon but click karo or seek tejao life mehdoor chalta. I've collected low Java podcasting. Mujalakta Bilkul Podcast, Sunenge. He definitely was. I don't biased to Laker, but my confused view he looks sunny podcast is domain the future podcast he future because log you know app swipe time pass We actually ruke entertaining cheese. Re

"Career Maduro Jana isley apne select Kia TRS clips Hindi OK OK OK subscribe karo Bella icon but click karo or seek tejao life mehdoor chalta. I've collected low Java podcasting. Mujalakta Bilkul Podcast, Sunenge. He definitely was. I don't biased to Laker, but my confused view he looks sunny podcast is domain the future podcast he future because log you know app swipe time pass We actually ruke entertaining cheese. Relax, relaxing dance. Knowledge. The whole long videos may or specially podcast majority detailing at the in fact podcast or talks. I hope. Audience Creator. But Sir young India main hamare India Co scientist engineer sachiya or bohari field dimagg hai Abhishek badi Khushi otiki startup culture or in fact mujhe lagta because boats are a college is my startup program to youth kadhana bhapar Jaya Gaya hai or as A Bola jare Barbara yoga 2020 two 1030 decade whose startup culture. India Ki hum service class recession service class AB job market. Golden opportunity, Golden envi

In [6]:
import azure.cognitiveservices.speech as speechsdk

# Replace with your own subscription key and service region (e.g., "westus").
subscription_key = "8d0afea35c1542b4b685e81608ad4e0b"
service_region = "centralindia"
audio_filename = "test.wav"  # Replace with the path to your audio file.

def transcribe_audio_with_diarization(subscription_key, service_region, audio_filename):
    # Create an instance of a speech config with specified subscription key and service region.
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)
    speech_config.speech_recognition_language = "en-US"  # Set the language for transcription

    # Enable detailed output with word-level timestamps
    speech_config.output_format = speechsdk.OutputFormat.Detailed

    # Specify the audio file to be transcribed.
    audio_input = speechsdk.AudioConfig(filename=audio_filename)

    # Create a recognizer with the given settings.
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

    # Full transcription with speaker diarization details
    full_transcription = []

    def recognized(evt):
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            detailed_result = evt.result
            for element in detailed_result.json:
                speaker_id = element.get("SpeakerId", "Unknown")
                text = element["Text"]
                start_time = element["Offset"] / 10_000_000  # Convert to seconds
                end_time = (element["Offset"] + element["Duration"]) / 10_000_000  # Convert to seconds
                full_transcription.append({
                    "speaker": speaker_id,
                    "start_time": start_time,
                    "end_time": end_time,
                    "text": text
                })
                print(f"Speaker {speaker_id} [{start_time}s - {end_time}s]: {text}")
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print("No speech could be recognized")

    def canceled(evt):
        print(f"CANCELED: {evt.reason}")
        done.set()

    def session_stopped(evt):
        print("Session stopped")
        done.set()

    # Connect callbacks to the recognizer events.
    speech_recognizer.recognized.connect(recognized)
    speech_recognizer.canceled.connect(canceled)
    speech_recognizer.session_stopped.connect(session_stopped)

    # Start transcription.
    print("Starting transcription...")
    done = False
    while not done:
        result_future = speech_recognizer.recognize_once_async()
        result = result_future.get()
        if result.reason == speechsdk.ResultReason.Canceled:
            done = True

    # Print final transcription in the desired format
    for segment in full_transcription:
        print(f"Speaker {segment['speaker']} [{segment['start_time']}s - {segment['end_time']}s]: {segment['text']}")

    return full_transcription

# Run the transcription function.
transcribe_audio_with_diarization(subscription_key, service_region, audio_filename)


Starting transcription...
Session stopped
Session stopped
No speech could be recognized
Session stopped
Session stopped


[]

In [7]:
# Dictionary mapping Hindi numerals to their English equivalents
hindi_numerals = {
    "ek": "1",
    "do": "2",
    "teen": "3",
    "char": "4",
    "paanch": "5",
    "chhah": "6",
    "saat": "7",
    "aath": "8",
    "nau": "9",
    "das": "10",
    "gyarah": "11",
    "barah": "12",
    "terah": "13",
    "chaudah": "14",
    "pandrah": "15",
    "solah": "16",
    "satrah": "17",
    "atharah": "18",
    "unnis": "19",
    "bees": "20",
    "ikkees": "21",
    "baees": "22",
    "teees": "23",
    "chabees": "26",
    "sattaees": "27",
    "atthaees": "28",
    "untis": "29",
    "tees": "30",
    "ikatties": "31",
    "battis": "32",
    "tettis": "33",
    "chattis": "34",
    "paitis": "35",
    "chattalis": "36",
    "saitalis": "37",
    "artalis": "38",
    "unchas": "39",
    "pachaas": "50",
    "ikyaavan": "51",
    "bawan": "52",
    "tirepan": "53",
    "chauvan": "54",
    "pachpan": "55",
    "chhappan": "56",
    "sattavan": "57",
    "atthawan": "58",
    "unsath": "59",
    "saath": "60",
    "iksaath": "61",
    "basaath": "62",
    "tirsaath": "63",
    "chausaath": "64",
    "painsaath": "65",
    "chhiyasaath": "66",
    "sarsaath": "67",
    "arasaath": "68",
    "unsattar": "69",
    "sattar": "70",
    "ikhasattar": "71",
    "baisattar": "72",
    "taisattar": "73",
    "chaurasi": "74",
    "pachattar": "75",
    "chhiyattar": "76",
    "sattattar": "77",
    "atthattar": "78",
    "unnasi": "79",
    "asi": "80",
    "ikiasi": "81",
    "baisi": "82",
    "tirsath": "83",
    "chaurasi": "84",
    "pachasi": "85",
    "chhiyasi": "86",
    "sathasi": "87",
    "athasi": "88",
    "ninyasi": "89",
    "nabbe": "90",
    "ikyanwe": "91",
    "banwe": "92",
    "tiryanwe": "93",
    "chauranawe": "94",
    "pachyanwe": "95",
    "chhiyanwe": "96",
    "satawanwe": "97",
    "athawanwe": "98",
    "ninanwe": "99",
    "sau": "100",
    "dhai sau": "250",
    "sava sau": "125",
    "aath sau": "800",
    "do sau": "200",
    "dhai hazar": "2500",
    "ek hazar": "1000",
    "das hazar": "10000",
    "ek lakh": "100000",
    "das lakh": "1000000",
    "ek karod": "10000000",
}

def custom_post_processing(text):
    # Replace Hindi numerals with their corresponding English mathematical equivalents
    for hindi_term, math_term in hindi_numerals.items():
        text = text.replace(hindi_term, math_term)
    return text

# Example usage
sample_text = "Mujhe das hazar rupaye chahiye aur dhai sau ka balance."
processed_text = custom_post_processing(sample_text)
print(processed_text)  # Output: Mujhe 10000 rupaye chahiye aur 250 ka balance.


Mujhe 10 hazar rupaye chahiye aur dhai 100 ka balance.


In [9]:
import azure.cognitiveservices.speech as speechsdk
import re

# Replace with your own subscription key and service region (e.g., "westus").
subscription_key = "8d0afea35c1542b4b685e81608ad4e0b"
service_region = "centralindia"
audio_filename = "India top 10 bank.wav"  # Replace with the path to your audio file.

# Dictionary mapping Hindi numerals to their English equivalents
hindi_numerals = {
    "ek hazar": "1000",
    "do hazar": "2000",
    "teen hazar": "3000",
    "char hazar": "4000",
    "paanch hazar": "5000",
    "chhah hazar": "6000",
    "saat hazar": "7000",
    "aath hazar": "8000",
    "nau hazar": "9000",
    "das hazar": "10000",
    "gyarah hazar": "11000",
    "barah hazar": "12000",
    "terah hazar": "13000",
    "chaudah hazar": "14000",
    "pandrah hazar": "15000",
    "solah hazar": "16000",
    "satrah hazar": "17000",
    "atharah hazar": "18000",
    "unnis hazar": "19000",
    "bees hazar": "20000",
    "ek lakh": "100000",
    "das lakh": "1000000",
    "ek karod": "10000000",
    "dhai sau": "250",
    "sava sau": "125",
    "aath sau": "800",
    "do sau": "200",
    "ek": "1",
    "do": "2",
    "teen": "3",
    "char": "4",
    "paanch": "5",
    "chhah": "6",
    "saat": "7",
    "aath": "8",
    "nau": "9",
    "das": "10",
    "gyarah": "11",
    "barah": "12",
    "terah": "13",
    "chaudah": "14",
    "pandrah": "15",
    "solah": "16",
    "satrah": "17",
    "atharah": "18",
    "unnis": "19",
    "bees": "20",
    "ikkees": "21",
    "baees": "22",
    "teees": "23",
    "chabees": "26",
    "sattaees": "27",
    "atthaees": "28",
    "untis": "29",
    "tees": "30",
    "ikatties": "31",
    "battis": "32",
    "tettis": "33",
    "chattis": "34",
    "paitis": "35",
    "chattalis": "36",
    "saitalis": "37",
    "artalis": "38",
    "unchas": "39",
    "pachaas": "50",
    "ikyaavan": "51",
    "bawan": "52",
    "tirepan": "53",
    "chauvan": "54",
    "pachpan": "55",
    "chhappan": "56",
    "sattavan": "57",
    "atthawan": "58",
    "unsath": "59",
    "saath": "60",
    "iksaath": "61",
    "basaath": "62",
    "tirsaath": "63",
    "chausaath": "64",
    "painsaath": "65",
    "chhiyasaath": "66",
    "sarsaath": "67",
    "arasaath": "68",
    "unsattar": "69",
    "sattar": "70",
    "ikhasattar": "71",
    "baisattar": "72",
    "taisattar": "73",
    "chaurasi": "74",
    "pachattar": "75",
    "chhiyattar": "76",
    "sattattar": "77",
    "atthattar": "78",
    "unnasi": "79",
    "asi": "80",
    "ikiasi": "81",
    "baisi": "82",
    "tirsath": "83",
    "chaurasi": "84",
    "pachasi": "85",
    "chhiyasi": "86",
    "sathasi": "87",
    "athasi": "88",
    "ninyasi": "89",
    "nabbe": "90",
    "ikyanwe": "91",
    "banye": "92",
    "tiryanwe": "93",
    "chauranawe": "94",
    "pachyanwe": "95",
    "chhiyanwe": "96",
    "satawanwe": "97",
    "athawanwe": "98",
    "ninanwe": "99",
    "sau": "100",
}

def custom_post_processing(text):
    # Replace Hindi numerals with their corresponding English mathematical equivalents
    for hindi_term, math_term in hindi_numerals.items():
        # Use word boundaries to replace only whole words
        text = re.sub(r'\b' + hindi_term + r'\b', math_term, text)
    return text

def transcribe_audio(subscription_key, service_region, audio_filename):
    # Create an instance of a speech config with specified subscription key and service region.
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)
    
    # Enable detailed output with word-level timestamps
    speech_config.output_format = speechsdk.OutputFormat.Detailed

    # Enable profanity filtering
    speech_config.set_profanity(speechsdk.ProfanityOption.Masked)

    # Specify the audio file to be transcribed.
    audio_input = speechsdk.AudioConfig(filename=audio_filename)

    # Create a recognizer with the given settings.
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

    # Full transcription with speaker diarization details
    full_transcription = []

    def recognized(evt):
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            detailed_result = evt.result.json
            speaker_id = detailed_result.get("SpeakerId", "Unknown")
            text = detailed_result["DisplayText"]
            text = custom_post_processing(text)
            start_time = detailed_result["Offset"] / 10_000_000  # Convert to seconds
            end_time = (detailed_result["Offset"] + detailed_result["Duration"]) / 10_000_000  # Convert to seconds
            full_transcription.append({
                "speaker": speaker_id,
                "start_time": start_time,
                "end_time": end_time,
                "text": text
            })
            print(f"Speaker {speaker_id} [{start_time}s - {end_time}s]: {text}")
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print("No speech could be recognized")

    def canceled(evt):
        print(f"CANCELED: {evt.reason}")
        done.set()

    def session_stopped(evt):
        print("Session stopped")
        done.set()

    # Connect callbacks to the recognizer events.
    speech_recognizer.recognized.connect(recognized)
    speech_recognizer.canceled.connect(canceled)
    speech_recognizer.session_stopped.connect(session_stopped)

    # Start transcription.
    print("Starting transcription...")
    done = False
    while not done:
        result_future = speech_recognizer.recognize_once_async()
        result = result_future.get()
        if result.reason == speechsdk.ResultReason.Canceled:
            done = True

    # Print final transcription in the desired format
    complete_transcription = '\n'.join([f"Speaker {seg['speaker']} [{seg['start_time']}s - {seg['end_time']}s]: {seg['text']}" for seg in full_transcription])
    print("Final transcription:", complete_transcription)
    return complete_transcription

# Run the transcription function.
transcribe_audio(subscription_key, service_region, audio_filename)


Starting transcription...
Session stopped
Session stopped
Session stopped
No speech could be recognized
Session stopped
Session stopped
No speech could be recognized
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
No speech could be recognized
Session stopped
No speech could be recognized
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
No speech could be recognized
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
No speech could be recognized
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
Session stopped
No speech could be recognized
Session stopped
Session stopped
No speech could be recognized
Session stopped
No speech coul

''

In [10]:
import re

# Dictionary mapping Hindi numerals to their English mathematical equivalents
hindi_numerals = {
    "ek": "1",
    "do": "2",
    "teen": "3",
    "char": "4",
    "paanch": "5",
    "chhah": "6",
    "saat": "7",
    "aath": "8",
    "nau": "9",
    "das": "10",
    "gyarah": "11",
    "barah": "12",
    "terah": "13",
    "chaudah": "14",
    "pandrah": "15",
    "solah": "16",
    "satrah": "17",
    "atharah": "18",
    "unnis": "19",
    "bees": "20",
    "ikkees": "21",
    "baees": "22",
    "teees": "23",
    "chabees": "26",
    "sattaees": "27",
    "atthaees": "28",
    "untis": "29",
    "tees": "30",
    "ikatties": "31",
    "battis": "32",
    "tettis": "33",
    "chattis": "34",
    "paitis": "35",
    "chattalis": "36",
    "saitalis": "37",
    "artalis": "38",
    "unchas": "39",
    "pachaas": "50",
    "ikyaavan": "51",
    "bawan": "52",
    "tirepan": "53",
    "chauvan": "54",
    "pachpan": "55",
    "chhappan": "56",
    "sattavan": "57",
    "atthawan": "58",
    "unsath": "59",
    "saath": "60",
    "iksaath": "61",
    "basaath": "62",
    "tirsaath": "63",
    "chausaath": "64",
    "painsaath": "65",
    "chhiyasaath": "66",
    "sarsaath": "67",
    "arasaath": "68",
    "unsattar": "69",
    "sattar": "70",
    "ikhasattar": "71",
    "baisattar": "72",
    "taisattar": "73",
    "chaurasi": "74",
    "pachattar": "75",
    "chhiyattar": "76",
    "sattattar": "77",
    "atthattar": "78",
    "unnasi": "79",
    "asi": "80",
    "ikiasi": "81",
    "baisi": "82",
    "tirsath": "83",
    "chaurasi": "84",
    "pachasi": "85",
    "chhiyasi": "86",
    "sathasi": "87",
    "athasi": "88",
    "ninyasi": "89",
    "nabbe": "90",
    "ikyanwe": "91",
    "banye": "92",
    "tiryanwe": "93",
    "chauranawe": "94",
    "pachyanwe": "95",
    "chhiyanwe": "96",
    "satawanwe": "97",
    "athawanwe": "98",
    "ninanwe": "99",
    "sau": "100",
    "dhai sau": "250",
    "sava sau": "125",
    "aath sau": "800",
    "do sau": "200",
    "dhai hazar": "2500",
    "ek hazar": "1000",
    "das hazar": "10000",
    "ek lakh": "100000",
    "das lakh": "1000000",
    "ek karod": "10000000",
}

def custom_post_processing(text):
    # Replace Hindi numerals with their corresponding English mathematical equivalents
    for hindi_term, math_term in hindi_numerals.items():
        text = re.sub(r'\b' + hindi_term + r'\b', math_term, text)
    return text

# Example input text containing Hindi numerals
input_text = "Mujhe das hazar rupaye chahiye aur dhai sau ka balance."

# Process the input text to convert Hindi numerals to mathematical equivalents
output_text = custom_post_processing(input_text)

# Print the output text
print("Input:", input_text)
print("Output:", output_text)


Input: Mujhe das hazar rupaye chahiye aur dhai sau ka balance.
Output: Mujhe 10 hazar rupaye chahiye aur dhai 100 ka balance.


In [12]:
import re

# Dictionary mapping Hindi numerals to their English mathematical equivalents
hindi_numerals = {
    "ek hazar": "1000",
    "dhai hazar": "2500",
    "das hazar": "10000",
    "ek lakh": "100000",
    "das lakh": "1000000",
    "ek karod": "10000000",
    "dhai sau": "250",
    "sava sau": "125",
    "aath sau": "800",
    "do sau": "200",
    "ek": "1",
    "do": "2",
    "teen": "3",
    "char": "4",
    "paanch": "5",
    "chhah": "6",
    "saat": "7",
    "aath": "8",
    "nau": "9",
    "das": "10",
    "gyarah": "11",
    "barah": "12",
    "terah": "13",
    "chaudah": "14",
    "pandrah": "15",
    "solah": "16",
    "satrah": "17",
    "atharah": "18",
    "unnis": "19",
    "bees": "20",
    "ikkees": "21",
    "baees": "22",
    "teees": "23",
    "chabees": "26",
    "sattaees": "27",
    "atthaees": "28",
    "untis": "29",
    "tees": "30",
    "ikatties": "31",
    "battis": "32",
    "tettis": "33",
    "chattis": "34",
    "paitis": "35",
    "chattalis": "36",
    "saitalis": "37",
    "artalis": "38",
    "unchas": "39",
    "pachaas": "50",
    "ikyaavan": "51",
    "bawan": "52",
    "tirepan": "53",
    "chauvan": "54",
    "pachpan": "55",
    "chhappan": "56",
    "sattavan": "57",
    "atthawan": "58",
    "unsath": "59",
    "saath": "60",
    "iksaath": "61",
    "basaath": "62",
    "tirsaath": "63",
    "chausaath": "64",
    "painsaath": "65",
    "chhiyasaath": "66",
    "sarsaath": "67",
    "arasaath": "68",
    "unsattar": "69",
    "sattar": "70",
    "ikhasattar": "71",
    "baisattar": "72",
    "taisattar": "73",
    "chaurasi": "74",
    "pachattar": "75",
    "chhiyattar": "76",
    "sattattar": "77",
    "atthattar": "78",
    "unnasi": "79",
    "asi": "80",
    "ikiasi": "81",
    "baisi": "82",
    "tirsath": "83",
    "chaurasi": "84",
    "pachasi": "85",
    "chhiyasi": "86",
    "sathasi": "87",
    "athasi": "88",
    "ninyasi": "89",
    "nabbe": "90",
    "ikyanwe": "91",
    "banye": "92",
    "tiryanwe": "93",
    "chauranawe": "94",
    "pachyanwe": "95",
    "chhiyanwe": "96",
    "satawanwe": "97",
    "athawanwe": "98",
    "ninanwe": "99",
    "sau": "100"
}

def custom_post_processing(text):
    # Replace Hindi numerals with their corresponding English mathematical equivalents
    sorted_numerals = sorted(hindi_numerals.keys(), key=lambda x: len(x), reverse=True)
    for hindi_term in sorted_numerals:
        math_term = hindi_numerals[hindi_term]
        text = re.sub(r'\b' + re.escape(hindi_term) + r'\b', math_term, text)
    return text

# Example input text containing Hindi numerals
input_text = "Mujhe das hazar rupaye chahiye aur sade teen sau ka balance."

# Process the input text to convert Hindi numerals to mathematical equivalents
output_text = custom_post_processing(input_text)

# Print the output text
print("Input:", input_text)
print("Output:", output_text)


Input: Mujhe das hazar rupaye chahiye aur sade teen sau ka balance.
Output: Mujhe 10000 rupaye chahiye aur sade 3 100 ka balance.


In [13]:
import re

# Extend the dictionary to include more complex numerals and fractions
hindi_numerals = {
    "ek": "1",
    "do": "2",
    "teen": "3",
    "char": "4",
    "paanch": "5",
    "chhah": "6",
    "saat": "7",
    "aath": "8",
    "nau": "9",
    "das": "10",
    "gyarah": "11",
    "barah": "12",
    "terah": "13",
    "chaudah": "14",
    "pandrah": "15",
    "solah": "16",
    "satrah": "17",
    "atharah": "18",
    "unnis": "19",
    "bees": "20",
    "ikkees": "21",
    "baees": "22",
    "teees": "23",
    "chaubees": "24",
    "pachchees": "25",
    "chhabees": "26",
    "sattaees": "27",
    "atthaees": "28",
    "untis": "29",
    "tees": "30",
    "ikatties": "31",
    "battis": "32",
    "tettis": "33",
    "chattis": "34",
    "paitis": "35",
    "chattalis": "36",
    "saitalis": "37",
    "artalis": "38",
    "unchas": "39",
    "pachaas": "50",
    "ikyaavan": "51",
    "bawan": "52",
    "tirepan": "53",
    "chauvan": "54",
    "pachpan": "55",
    "chhappan": "56",
    "sattavan": "57",
    "atthawan": "58",
    "unsath": "59",
    "saath": "60",
    "iksaath": "61",
    "basaath": "62",
    "tirsaath": "63",
    "chausaath": "64",
    "painsaath": "65",
    "chhiyasaath": "66",
    "sarsaath": "67",
    "arasaath": "68",
    "unsattar": "69",
    "sattar": "70",
    "ikhasattar": "71",
    "baisattar": "72",
    "taisattar": "73",
    "chaurasi": "74",
    "pachattar": "75",
    "chhiyattar": "76",
    "sattattar": "77",
    "atthattar": "78",
    "unnasi": "79",
    "asi": "80",
    "ikiasi": "81",
    "baisi": "82",
    "tiryanwe": "93",
    "chauranawe": "94",
    "pachyanwe": "95",
    "chhiyanwe": "96",
    "satawanwe": "97",
    "athawanwe": "98",
    "ninanwe": "99",
    "sau": "100",
    "dhai": "2.5",
    "sava": "1.25",
    "aadha": "0.5",
    "paune": "0.75",
}

# Patterns for detecting larger numerals
large_numerals = {
    "sau": 100,
    "hazaar": 1000,
    "lakh": 100000,
    "karod": 10000000
}

def convert_large_numerals(match):
    parts = match.group(0).split()
    number = hindi_numerals.get(parts[0], parts[0])
    if len(parts) > 1 and parts[1] in large_numerals:
        number = str(float(number) * large_numerals[parts[1]])
    return number

def custom_post_processing(text):
    # Replace Hindi numerals with their corresponding English mathematical equivalents
    sorted_numerals = sorted(hindi_numerals.keys(), key=lambda x: len(x), reverse=True)
    
    # Handle large numerals first
    pattern = re.compile(r'\b(' + '|'.join(hindi_numerals.keys()) + r')\s(hazaar|lakh|karod)\b')
    text = pattern.sub(convert_large_numerals, text)
    
    # Handle all other numerals
    for hindi_term in sorted_numerals:
        math_term = hindi_numerals[hindi_term]
        text = re.sub(r'\b' + re.escape(hindi_term) + r'\b', math_term, text)
    
    return text

# Example input text containing Hindi numerals
input_text = "Mujhe das hazaar rupaye chahiye aur dhai sau ka balance aur paune ek hazaar."

# Process the input text to convert Hindi numerals to mathematical equivalents
output_text = custom_post_processing(input_text)

# Print the output text
print("Input:", input_text)
print("Output:", output_text)


Input: Mujhe das hazaar rupaye chahiye aur dhai sau ka balance aur paune ek hazaar.
Output: Mujhe 10000.0 rupaye chahiye aur 2.5 100 ka balance aur 0.75 1000.0.


In [15]:
import re

# Extended dictionary for Hindi numerals
hindi_numerals = {
    "ek": "1",
    "do": "2",
    "teen": "3",
    "char": "4",
    "paanch": "5",
    "chhah": "6",
    "saat": "7",
    "aath": "8",
    "nau": "9",
    "das": "10",
    "gyarah": "11",
    "barah": "12",
    "terah": "13",
    "chaudah": "14",
    "pandrah": "15",
    "solah": "16",
    "satrah": "17",
    "atharah": "18",
    "unnis": "19",
    "bees": "20",
    "ikkees": "21",
    "baees": "22",
    "teees": "23",
    "chaubees": "24",
    "pachchees": "25",
    "chhabees": "26",
    "sattaees": "27",
    "atthaees": "28",
    "untis": "29",
    "tees": "30",
    "ikatties": "31",
    "battis": "32",
    "tettis": "33",
    "chattis": "34",
    "paitis": "35",
    "chattalis": "36",
    "saitalis": "37",
    "artalis": "38",
    "unchas": "39",
    "pachaas": "50",
    "ikyaavan": "51",
    "bawan": "52",
    "tirepan": "53",
    "chauvan": "54",
    "pachpan": "55",
    "chhappan": "56",
    "sattavan": "57",
    "atthawan": "58",
    "unsath": "59",
    "saath": "60",
    "iksaath": "61",
    "basaath": "62",
    "tirsaath": "63",
    "chausaath": "64",
    "painsaath": "65",
    "chhiyasaath": "66",
    "sarsaath": "67",
    "arasaath": "68",
    "unsattar": "69",
    "sattar": "70",
    "ikhasattar": "71",
    "baisattar": "72",
    "taisattar": "73",
    "chaurasi": "74",
    "pachattar": "75",
    "chhiyattar": "76",
    "sattattar": "77",
    "atthattar": "78",
    "unnasi": "79",
    "asi": "80",
    "ikiasi": "81",
    "baisi": "82",
    "tiryanwe": "93",
    "chauranawe": "94",
    "pachyanwe": "95",
    "chhiyanwe": "96",
    "satawanwe": "97",
    "athawanwe": "98",
    "ninanwe": "99",
    "sau": "100",
    "dhai": "2.5",
    "sava": "1.25",
    "aadha": "0.5",
    "paune": "0.75",
}

# Large numerals handling
large_numerals = {
    "sau": 100,
    "hazaar": 1000,
    "lakh": 100000,
    "karod": 10000000
}

def convert_large_numerals(match):
    number_word = match.group(1)
    large_unit = match.group(2)
    
    # Handle fractions like "dhai hazaar" or "paune ek hazaar"
    if number_word in hindi_numerals:
        number = hindi_numerals[number_word]
    else:
        number = str(eval(number_word))

    if large_unit in large_numerals:
        number = str(float(number) * large_numerals[large_unit])
    
    return number

def custom_post_processing(text):
    # Handle large numerals first
    pattern = re.compile(r'\b([a-zA-Z ]+?)\s(hazaar|lakh|karod|sau)\b')
    text = pattern.sub(convert_large_numerals, text)
    
    # Replace Hindi numerals with their corresponding English mathematical equivalents
    sorted_numerals = sorted(hindi_numerals.keys(), key=lambda x: len(x), reverse=True)
    
    for hindi_term in sorted_numerals:
        math_term = hindi_numerals[hindi_term]
        text = re.sub(r'\b' + re.escape(hindi_term) + r'\b', math_term, text)
    
    return text

# Example input text containing Hindi numerals
input_text = "Mujhe dash hazaar rupaye chahiye aur dhai sau ka balance aur paune ek hazaar."

# Process the input text to convert Hindi numerals to mathematical equivalents
output_text = custom_post_processing(input_text)

# Print the output text
print("Input:", input_text)
print("Output:", output_text)


SyntaxError: invalid syntax (<string>, line 1)

In [16]:
import re

# Extended dictionary for Hindi numerals
hindi_numerals = {
    "ek": "1",
    "do": "2",
    "teen": "3",
    "char": "4",
    "paanch": "5",
    "chhah": "6",
    "saat": "7",
    "aath": "8",
    "nau": "9",
    "das": "10",
    "gyarah": "11",
    "barah": "12",
    "terah": "13",
    "chaudah": "14",
    "pandrah": "15",
    "solah": "16",
    "satrah": "17",
    "atharah": "18",
    "unnis": "19",
    "bees": "20",
    "ikkees": "21",
    "baees": "22",
    "teees": "23",
    "chaubees": "24",
    "pachchees": "25",
    "chhabees": "26",
    "sattaees": "27",
    "atthaees": "28",
    "untis": "29",
    "tees": "30",
    "ikatties": "31",
    "battis": "32",
    "tettis": "33",
    "chattis": "34",
    "paitis": "35",
    "chattalis": "36",
    "saitalis": "37",
    "artalis": "38",
    "unchas": "39",
    "pachaas": "50",
    "ikyaavan": "51",
    "bawan": "52",
    "tirepan": "53",
    "chauvan": "54",
    "pachpan": "55",
    "chhappan": "56",
    "sattavan": "57",
    "atthawan": "58",
    "unsath": "59",
    "saath": "60",
    "iksaath": "61",
    "basaath": "62",
    "tirsaath": "63",
    "chausaath": "64",
    "painsaath": "65",
    "chhiyasaath": "66",
    "sarsaath": "67",
    "arasaath": "68",
    "unsattar": "69",
    "sattar": "70",
    "ikhasattar": "71",
    "baisattar": "72",
    "taisattar": "73",
    "chaurasi": "74",
    "pachattar": "75",
    "chhiyattar": "76",
    "sattattar": "77",
    "atthattar": "78",
    "unnasi": "79",
    "asi": "80",
    "ikiasi": "81",
    "baisi": "82",
    "tiryanwe": "93",
    "chauranawe": "94",
    "pachyanwe": "95",
    "chhiyanwe": "96",
    "satawanwe": "97",
    "athawanwe": "98",
    "ninanwe": "99",
    "sau": "100",
    "dhai": "2.5",
    "sava": "1.25",
    "aadha": "0.5",
    "paune": "0.75",
}

# Large numerals handling
large_numerals = {
    "sau": 100,
    "hazaar": 1000,
    "lakh": 100000,
    "karod": 10000000
}

def convert_large_numerals(match):
    number_word = match.group(1)
    large_unit = match.group(2)
    
    # Handle fractions like "dhai hazaar" or "paune ek hazaar"
    if number_word in hindi_numerals:
        number = hindi_numerals[number_word]
    else:
        try:
            number = str(eval(number_word))
        except SyntaxError:
            return match.group(0)  # Return original if cannot evaluate
    
    if large_unit in large_numerals:
        number = str(float(number) * large_numerals[large_unit])
    
    return number

def custom_post_processing(text):
    # Handle large numerals first
    pattern = re.compile(r'\b([a-zA-Z ]+?)\s(hazaar|lakh|karod|sau)\b')
    text = pattern.sub(convert_large_numerals, text)
    
    # Replace Hindi numerals with their corresponding English mathematical equivalents
    sorted_numerals = sorted(hindi_numerals.keys(), key=lambda x: len(x), reverse=True)
    
    for hindi_term in sorted_numerals:
        math_term = hindi_numerals[hindi_term]
        text = re.sub(r'\b' + re.escape(hindi_term) + r'\b', math_term, text)
    
    return text

# Example input text containing Hindi numerals
input_text = "Mujhe das hazaar rupaye chahiye aur dhai sau ka balance aur paune ek hazaar."

# Process the input text to convert Hindi numerals to mathematical equivalents
output_text = custom_post_processing(input_text)

# Print the output text
print("Input:", input_text)
print("Output:", output_text)


Input: Mujhe das hazaar rupaye chahiye aur dhai sau ka balance aur paune ek hazaar.
Output: Mujhe 10 hazaar rupaye chahiye aur 2.5 100 ka balance aur 0.75 1 hazaar.


In [17]:
import re

# Extended dictionary for Hindi numerals
hindi_numerals = {
    "ek": "1",
    "do": "2",
    "teen": "3",
    "char": "4",
    "paanch": "5",
    "chhah": "6",
    "saat": "7",
    "aath": "8",
    "nau": "9",
    "das": "10",
    "gyarah": "11",
    "barah": "12",
    "terah": "13",
    "chaudah": "14",
    "pandrah": "15",
    "solah": "16",
    "satrah": "17",
    "atharah": "18",
    "unnis": "19",
    "bees": "20",
    "ikkees": "21",
    "baees": "22",
    "teees": "23",
    "chaubees": "24",
    "pachchees": "25",
    "chhabees": "26",
    "sattaees": "27",
    "atthaees": "28",
    "untis": "29",
    "tees": "30",
    "ikatties": "31",
    "battis": "32",
    "tettis": "33",
    "chattis": "34",
    "paitis": "35",
    "chattalis": "36",
    "saitalis": "37",
    "artalis": "38",
    "unchas": "39",
    "pachaas": "50",
    "ikyaavan": "51",
    "bawan": "52",
    "tirepan": "53",
    "chauvan": "54",
    "pachpan": "55",
    "chhappan": "56",
    "sattavan": "57",
    "atthawan": "58",
    "unsath": "59",
    "saath": "60",
    "iksaath": "61",
    "basaath": "62",
    "tirsaath": "63",
    "chausaath": "64",
    "painsaath": "65",
    "chhiyasaath": "66",
    "sarsaath": "67",
    "arasaath": "68",
    "unsattar": "69",
    "sattar": "70",
    "ikhasattar": "71",
    "baisattar": "72",
    "taisattar": "73",
    "chaurasi": "74",
    "pachattar": "75",
    "chhiyattar": "76",
    "sattattar": "77",
    "atthattar": "78",
    "unnasi": "79",
    "asi": "80",
    "ikiasi": "81",
    "baisi": "82",
    "tiryanwe": "93",
    "chauranawe": "94",
    "pachyanwe": "95",
    "chhiyanwe": "96",
    "satawanwe": "97",
    "athawanwe": "98",
    "ninanwe": "99",
    "sau": "100",
    "dhai": "2.5",
    "sava": "1.25",
    "aadha": "0.5",
    "paune": "0.75",
}

# Large numerals handling
large_numerals = {
    "sau": 100,
    "hazaar": 1000,
    "lakh": 100000,
    "karod": 10000000
}

def convert_large_numerals(match):
    number_word = match.group(1)
    large_unit = match.group(2)
    
    # Handle fractions like "dhai hazaar" or "paune ek hazaar"
    if number_word in hindi_numerals:
        number = hindi_numerals[number_word]
    else:
        try:
            number = str(eval(number_word))
        except SyntaxError:
            return match.group(0)  # Return original if cannot evaluate
    
    if large_unit in large_numerals:
        number = str(float(number) * large_numerals[large_unit])
    
    return number

def custom_post_processing(text):
    # Handle large numerals first
    pattern = re.compile(r'\b([a-zA-Z ]+?)\s(hazaar|lakh|karod|sau)\b')
    text = pattern.sub(convert_large_numerals, text)
    
    # Replace Hindi numerals with their corresponding English mathematical equivalents
    sorted_numerals = sorted(hindi_numerals.keys(), key=lambda x: len(x), reverse=True)
    
    for hindi_term in sorted_numerals:
        math_term = hindi_numerals[hindi_term]
        text = re.sub(r'\b' + re.escape(hindi_term) + r'\b', math_term, text)
    
    return text

# Example input text containing Hindi numerals
input_text = "Mujhe das hazaar rupaye chahiye aur dhai sau ka balance aur paune ek hazaar."

# Process the input text to convert Hindi numerals to mathematical equivalents
output_text = custom_post_processing(input_text)

# Print the output text
print("Input:", input_text)
print("Output:", output_text)


Input: Mujhe das hazaar rupaye chahiye aur dhai sau ka balance aur paune ek hazaar.
Output: Mujhe 10 hazaar rupaye chahiye aur 2.5 100 ka balance aur 0.75 1 hazaar.


In [18]:
import re

# Extended dictionary for Hindi numerals
hindi_numerals = {
    "ek": "1",
    "do": "2",
    "teen": "3",
    "char": "4",
    "paanch": "5",
    "chhah": "6",
    "saat": "7",
    "aath": "8",
    "nau": "9",
    "das": "10",
    # Add more numerals as needed
}

# Large numerals handling
large_numerals = {
    "sau": 100,
    "hazaar": 1000,
    "lakh": 100000,
    "karod": 10000000
}

def convert_large_numerals(match):
    number_word = match.group(1)
    large_unit = match.group(2)
    
    # Handle fractions like "dhai hazaar" or "paune ek hazaar"
    if number_word in hindi_numerals:
        number = hindi_numerals[number_word]
    else:
        number = "0"  # Temporary solution
    
    if large_unit in large_numerals:
        number = str(float(number) * large_numerals[large_unit])
    
    return number

def custom_post_processing(text):
    # Handle large numerals first
    pattern = re.compile(r'\b([a-zA-Z ]+?)\s(hazaar|lakh|karod|sau)\b')
    text = pattern.sub(convert_large_numerals, text)
    
    # Replace Hindi numerals with their corresponding English mathematical equivalents
    sorted_numerals = sorted(hindi_numerals.keys(), key=lambda x: len(x), reverse=True)
    
    for hindi_term in sorted_numerals:
        math_term = hindi_numerals[hindi_term]
        text = re.sub(r'\b' + re.escape(hindi_term) + r'\b', math_term, text)
    
    return text

# Example input text containing Hindi numerals
input_text = "Mujhe chaualise hzaar char sao teen ruppey chahiye."

# Process the input text to convert Hindi numerals to mathematical equivalents
output_text = custom_post_processing(input_text)

# Print the output text
print("Input:", input_text)
print("Output:", output_text)


Input: Mujhe chaualise hzaar char sao teen ruppey chahiye.
Output: Mujhe chaualise hzaar 4 sao 3 ruppey chahiye.


In [19]:
import re

# Dictionary for Hindi numerals
hindi_numerals = {
    "ek": "1",
    "do": "2",
    "teen": "3",
    "char": "4",
    "paanch": "5",
    "chhah": "6",
    "saat": "7",
    "aath": "8",
    "nau": "9",
    "das": "10",
    "gyarah": "11",
    "barah": "12",
    "terah": "13",
    "chaudah": "14",
    "pandrah": "15",
    "solah": "16",
    "satrah": "17",
    "atharah": "18",
    "unnis": "19",
    "bees": "20",
    "ikkees": "21",
    "baees": "22",
    "teees": "23",
    "chaubees": "24",
    "pachchees": "25",
    "chhabees": "26",
    "sattaees": "27",
    "atthaees": "28",
    "untis": "29",
    "tees": "30",
    "ikatties": "31",
    "battis": "32",
    "tettis": "33",
    "chattis": "34",
    "paitis": "35",
    "chattalis": "36",
    "saitalis": "37",
    "artalis": "38",
    "unchas": "39",
    "pachaas": "50",
    "ikyaavan": "51",
    "bawan": "52",
    "tirepan": "53",
    "chauvan": "54",
    "pachpan": "55",
    "chhappan": "56",
    "sattavan": "57",
    "atthawan": "58",
    "unsath": "59",
    "saath": "60",
    "iksaath": "61",
    "basaath": "62",
    "tirsaath": "63",
    "chausaath": "64",
    "painsaath": "65",
    "chhiyasaath": "66",
    "sarsaath": "67",
    "arasaath": "68",
    "unsattar": "69",
    "sattar": "70",
    "ikhasattar": "71",
    "baisattar": "72",
    "taisattar": "73",
    "chaurasi": "74",
    "pachattar": "75",
    "chhiyattar": "76",
    "sattattar": "77",
    "atthattar": "78",
    "unnasi": "79",
    "asi": "80",
    "ikiasi": "81",
    "baisi": "82",
    "ninyasi": "89",
    "nabbe": "90",
    "ikyanwe": "91",
    "banwe": "92",
    "tiryanwe": "93",
    "chauranawe": "94",
    "pachyanwe": "95",
    "chhiyanwe": "96",
    "satawanwe": "97",
    "athawanwe": "98",
    "ninanwe": "99",
    "sau": "100",
    "dhai": "2.5",
    "sava": "1.25",
    "aadha": "0.5",
    "paune": "0.75",
}

# Dictionary for larger units
large_units = {
    "hazaar": 1000,
    "lakh": 100000,
    "karod": 10000000,
    "crore": 100000000,
    # Add more units as needed
}

def convert_hindi_numerals(text):
    # Regular expression to match Hindi numerals
    pattern = re.compile(r'\b(' + '|'.join(hindi_numerals.keys()) + r')\b', re.IGNORECASE)
    
    def replace(match):
        return hindi_numerals.get(match.group(0).lower(), match.group(0))

    # Replace Hindi numerals with their numerical equivalents
    text = pattern.sub(replace, text)
    
    # Handle larger units
    for unit, value in large_units.items():
        pattern = re.compile(r'\b(\d+)\s*(' + re.escape(unit) + r')\b', re.IGNORECASE)
        text = pattern.sub(lambda x: str(int(x.group(1)) * value), text)

    return text

# Example input text containing Hindi numerals
input_text = "Mujhe chaualise hazaar char sau teen ruppey chahiye."

# Process the input text to convert Hindi numerals to numerical equivalents
output_text = convert_hindi_numerals(input_text)

# Print the output text
print("Input:", input_text)
print("Output:", output_text)


Input: Mujhe chaualise hazaar char sau teen ruppey chahiye.
Output: Mujhe chaualise hazaar 4 100 3 ruppey chahiye.


In [20]:
import re

# Dictionary for Hindi numerals
hindi_numerals = {
    "ek": "1",
    "do": "2",
    "teen": "3",
    "char": "4",
    "paanch": "5",
    "chhah": "6",
    "saat": "7",
    "aath": "8",
    "nau": "9",
    "das": "10",
    "gyarah": "11",
    "barah": "12",
    "terah": "13",
    "chaudah": "14",
    "pandrah": "15",
    "solah": "16",
    "satrah": "17",
    "atharah": "18",
    "unnis": "19",
    "bees": "20",
    "ikkees": "21",
    "baees": "22",
    "teees": "23",
    "chaubees": "24",
    "pachchees": "25",
    "chhabees": "26",
    "sattaees": "27",
    "atthaees": "28",
    "untis": "29",
    "tees": "30",
    "ikatties": "31",
    "battis": "32",
    "tettis": "33",
    "chattis": "34",
    "paitis": "35",
    "chattalis": "36",
    "saitalis": "37",
    "artalis": "38",
    "unchas": "39",
    "pachaas": "50",
    "ikyaavan": "51",
    "bawan": "52",
    "tirepan": "53",
    "chauvan": "54",
    "pachpan": "55",
    "chhappan": "56",
    "sattavan": "57",
    "atthawan": "58",
    "unsath": "59",
    "saath": "60",
    "iksaath": "61",
    "basaath": "62",
    "tirsaath": "63",
    "chausaath": "64",
    "painsaath": "65",
    "chhiyasaath": "66",
    "sarsaath": "67",
    "arasaath": "68",
    "unsattar": "69",
    "sattar": "70",
    "ikhasattar": "71",
    "baisattar": "72",
    "taisattar": "73",
    "chaurasi": "74",
    "pachattar": "75",
    "chhiyattar": "76",
    "sattattar": "77",
    "atthattar": "78",
    "unnasi": "79",
    "asi": "80",
    "ikiasi": "81",
    "baisi": "82",
    "ninyasi": "89",
    "nabbe": "90",
    "ikyanwe": "91",
    "banwe": "92",
    "tiryanwe": "93",
    "chauranawe": "94",
    "pachyanwe": "95",
    "chhiyanwe": "96",
    "satawanwe": "97",
    "athawanwe": "98",
    "ninanwe": "99",
    "sau": "100",
    "dhai": "2.5",
    "sava": "1.25",
    "aadha": "0.5",
    "paune": "0.75",
}

# Dictionary for larger units
large_units = {
    "hazaar": 1000,
    "lakh": 100000,
    "karod": 10000000,
    # Add more units as needed
}

def convert_hindi_numerals(text):
    # Regular expression to match Hindi numerals
    pattern = re.compile(r'\b(' + '|'.join(hindi_numerals.keys()) + r')\b', re.IGNORECASE)
    
    def replace(match):
        return hindi_numerals.get(match.group(0).lower(), match.group(0))

    # Replace Hindi numerals with their numerical equivalents
    text = pattern.sub(replace, text)
    
    # Handle larger units
    for unit, value in large_units.items():
        pattern = re.compile(r'\b(\d+)\s*(' + re.escape(unit) + r')\b', re.IGNORECASE)
        text = pattern.sub(lambda x: str(int(x.group(1)) * value), text)

    return text

# Function to evaluate the numerical expression
def evaluate_expression(expression):
    return eval(expression)

# Example input text containing Hindi numerals
input_text = "Mujhe chaualise hazaar char sau teen ruppey chahiye."

# Convert Hindi numerals to numerical equivalents
converted_text = convert_hindi_numerals(input_text)

# Evaluate the expression
result = evaluate_expression(converted_text)

# Print the output text
print("Input:", input_text)
print("Converted Text:", converted_text)
print("Result:", result)


SyntaxError: invalid syntax (<string>, line 1)

In [21]:
import re

# Dictionary for Hindi numerals
hindi_numerals = {
    "ek": "1",
    "do": "2",
    "teen": "3",
    "char": "4",
    "paanch": "5",
    "chhah": "6",
    "saat": "7",
    "aath": "8",
    "nau": "9",
    "das": "10",
    "gyarah": "11",
    "barah": "12",
    "terah": "13",
    "chaudah": "14",
    "pandrah": "15",
    "solah": "16",
    "satrah": "17",
    "atharah": "18",
    "unnis": "19",
    "bees": "20",
    "ikkees": "21",
    "baees": "22",
    "teees": "23",
    "chaubees": "24",
    "pachchees": "25",
    "chhabees": "26",
    "sattaees": "27",
    "atthaees": "28",
    "untis": "29",
    "tees": "30",
    "ikatties": "31",
    "battis": "32",
    "tettis": "33",
    "chattis": "34",
    "paitis": "35",
    "chattalis": "36",
    "saitalis": "37",
    "artalis": "38",
    "unchas": "39",
    "pachaas": "50",
    "ikyaavan": "51",
    "bawan": "52",
    "tirepan": "53",
    "chauvan": "54",
    "pachpan": "55",
    "chhappan": "56",
    "sattavan": "57",
    "atthawan": "58",
    "unsath": "59",
    "saath": "60",
    "iksaath": "61",
    "basaath": "62",
    "tirsaath": "63",
    "chausaath": "64",
    "painsaath": "65",
    "chhiyasaath": "66",
    "sarsaath": "67",
    "arasaath": "68",
    "unsattar": "69",
    "sattar": "70",
    "ikhasattar": "71",
    "baisattar": "72",
    "taisattar": "73",
    "chaurasi": "74",
    "pachattar": "75",
    "chhiyattar": "76",
    "sattattar": "77",
    "atthattar": "78",
    "unnasi": "79",
    "asi": "80",
    "ikiasi": "81",
    "baisi": "82",
    "ninyasi": "89",
    "nabbe": "90",
    "ikyanwe": "91",
    "banwe": "92",
    "tiryanwe": "93",
    "chauranawe": "94",
    "pachyanwe": "95",
    "chhiyanwe": "96",
    "satawanwe": "97",
    "athawanwe": "98",
    "ninanwe": "99",
    "sau": "100",
    "dhai": "2.5",
    "sava": "1.25",
    "aadha": "0.5",
    "paune": "0.75",
}

# Dictionary for larger units
large_units = {
    "hazaar": 1000,
    "lakh": 100000,
    "karod": 10000000,
    # Add more units as needed
}

def convert_hindi_numerals(text):
    # Regular expression to match Hindi numerals
    pattern = re.compile(r'\b(' + '|'.join(hindi_numerals.keys()) + r')\b', re.IGNORECASE)
    
    def replace(match):
        return hindi_numerals.get(match.group(0).lower(), match.group(0))

    # Replace Hindi numerals with their numerical equivalents
    text = pattern.sub(replace, text)
    
    # Handle larger units
    for unit, value in large_units.items():
        pattern = re.compile(r'\b(\d+)\s*(' + re.escape(unit) + r')\b', re.IGNORECASE)
        text = pattern.sub(lambda x: str(int(x.group(1)) * value), text)

    return text

# Function to evaluate the numerical expression
def evaluate_expression(expression):
    return eval(expression.replace(' ', '+'))

# Example input text containing Hindi numerals
input_text = "Mujhe chaualise hazaar char sau teen ruppey chahiye."

# Convert Hindi numerals to numerical equivalents
converted_text = convert_hindi_numerals(input_text)

# Evaluate the expression
result = evaluate_expression(converted_text)

# Print the output text
print("Input:", input_text)
print("Converted Text:", converted_text)
print("Result:", result)


SyntaxError: invalid syntax (<string>, line 1)

In [22]:
import re

# Dictionary for Hindi numerals
hindi_numerals = {
    "ek": "1",
    "do": "2",
    "teen": "3",
    "char": "4",
    "paanch": "5",
    "chhah": "6",
    "saat": "7",
    "aath": "8",
    "nau": "9",
    "das": "10",
    "gyarah": "11",
    "barah": "12",
    "terah": "13",
    "chaudah": "14",
    "pandrah": "15",
    "solah": "16",
    "satrah": "17",
    "atharah": "18",
    "unnis": "19",
    "bees": "20",
    "ikkees": "21",
    "baees": "22",
    "teees": "23",
    "chaubees": "24",
    "pachchees": "25",
    "chhabees": "26",
    "sattaees": "27",
    "atthaees": "28",
    "untis": "29",
    "tees": "30",
    "ikatties": "31",
    "battis": "32",
    "tettis": "33",
    "chattis": "34",
    "paitis": "35",
    "chattalis": "36",
    "saitalis": "37",
    "artalis": "38",
    "unchas": "39",
    "pachaas": "50",
    "ikyaavan": "51",
    "bawan": "52",
    "tirepan": "53",
    "chauvan": "54",
    "pachpan": "55",
    "chhappan": "56",
    "sattavan": "57",
    "atthawan": "58",
    "unsath": "59",
    "saath": "60",
    "iksaath": "61",
    "basaath": "62",
    "tirsaath": "63",
    "chausaath": "64",
    "painsaath": "65",
    "chhiyasaath": "66",
    "sarsaath": "67",
    "arasaath": "68",
    "unsattar": "69",
    "sattar": "70",
    "ikhasattar": "71",
    "baisattar": "72",
    "taisattar": "73",
    "chaurasi": "74",
    "pachattar": "75",
    "chhiyattar": "76",
    "sattattar": "77",
    "atthattar": "78",
    "unnasi": "79",
    "asi": "80",
    "ikiasi": "81",
    "baisi": "82",
    "ninyasi": "89",
    "nabbe": "90",
    "ikyanwe": "91",
    "banwe": "92",
    "tiryanwe": "93",
    "chauranawe": "94",
    "pachyanwe": "95",
    "chhiyanwe": "96",
    "satawanwe": "97",
    "athawanwe": "98",
    "ninanwe": "99",
    "sau": "100",
    "dhai": "2.5",
    "sava": "1.25",
    "aadha": "0.5",
    "paune": "0.75",
}

# Dictionary for larger units
large_units = {
    "hazaar": 1000,
    "lakh": 100000,
    "karod": 10000000,
    # Add more units as needed
}

def convert_hindi_numerals(text):
    # Regular expression to match Hindi numerals
    pattern = re.compile(r'\b(' + '|'.join(hindi_numerals.keys()) + r')\b', re.IGNORECASE)
    
    def replace(match):
        return hindi_numerals.get(match.group(0).lower(), match.group(0))

    # Replace Hindi numerals with their numerical equivalents
    text = pattern.sub(replace, text)
    
    # Handle larger units
    for unit, value in large_units.items():
        pattern = re.compile(r'\b(\d+)\s*(' + re.escape(unit) + r')\b', re.IGNORECASE)
        text = pattern.sub(lambda x: str(int(x.group(1)) * value), text)

    return text

# Function to evaluate the numerical expression
def evaluate_expression(expression):
    expression = expression.replace(' ', '+')
    return eval(expression)

# Example input text containing Hindi numerals
input_text = "Mujhe chaualise hazaar char sau teen ruppey chahiye."

# Convert Hindi numerals to numerical equivalents
converted_text = convert_hindi_numerals(input_text)

# Evaluate the expression
result = evaluate_expression(converted_text)

# Print the output text
print("Input:", input_text)
print("Converted Text:", converted_text)
print("Result:", result)



SyntaxError: invalid syntax (<string>, line 1)

In [23]:
import re

# Dictionary for Hindi numerals
hindi_numerals = {
    "ek": "1",
    "do": "2",
    "teen": "3",
    "char": "4",
    "paanch": "5",
    "chhah": "6",
    "saat": "7",
    "aath": "8",
    "nau": "9",
    "das": "10",
    "gyarah": "11",
    "barah": "12",
    "terah": "13",
    "chaudah": "14",
    "pandrah": "15",
    "solah": "16",
    "satrah": "17",
    "atharah": "18",
    "unnis": "19",
    "bees": "20",
    "ikkees": "21",
    "baees": "22",
    "teees": "23",
    "chaubees": "24",
    "pachchees": "25",
    "chhabees": "26",
    "sattaees": "27",
    "atthaees": "28",
    "untis": "29",
    "tees": "30",
    "ikatties": "31",
    "battis": "32",
    "tettis": "33",
    "chattis": "34",
    "paitis": "35",
    "chattalis": "36",
    "saitalis": "37",
    "artalis": "38",
    "unchas": "39",
    "pachaas": "50",
    "ikyaavan": "51",
    "bawan": "52",
    "tirepan": "53",
    "chauvan": "54",
    "pachpan": "55",
    "chhappan": "56",
    "sattavan": "57",
    "atthawan": "58",
    "unsath": "59",
    "saath": "60",
    "iksaath": "61",
    "basaath": "62",
    "tirsaath": "63",
    "chausaath": "64",
    "painsaath": "65",
    "chhiyasaath": "66",
    "sarsaath": "67",
    "arasaath": "68",
    "unsattar": "69",
    "sattar": "70",
    "ikhasattar": "71",
    "baisattar": "72",
    "taisattar": "73",
    "chaurasi": "74",
    "pachattar": "75",
    "chhiyattar": "76",
    "sattattar": "77",
    "atthattar": "78",
    "unnasi": "79",
    "asi": "80",
    "ikiasi": "81",
    "baisi": "82",
    "ninyasi": "89",
    "nabbe": "90",
    "ikyanwe": "91",
    "banwe": "92",
    "tiryanwe": "93",
    "chauranawe": "94",
    "pachyanwe": "95",
    "chhiyanwe": "96",
    "satawanwe": "97",
    "athawanwe": "98",
    "ninanwe": "99",
    "sau": "100",
    "dhai": "2.5",
    "sava": "1.25",
    "aadha": "0.5",
    "paune": "0.75",
}

# Dictionary for larger units
large_units = {
    "hazaar": 1000,
    "lakh": 100000,
    "karod": 10000000,
    # Add more units as needed
}

def convert_hindi_numerals(text):
    # Regular expression to match Hindi numerals
    pattern = re.compile(r'\b(' + '|'.join(hindi_numerals.keys()) + r')\b', re.IGNORECASE)
    
    def replace(match):
        return hindi_numerals.get(match.group(0).lower(), match.group(0))

    # Replace Hindi numerals with their numerical equivalents
    text = pattern.sub(replace, text)
    
    # Handle larger units
    for unit, value in large_units.items():
        pattern = re.compile(r'\b(\d+)\s*(' + re.escape(unit) + r')\b', re.IGNORECASE)
        text = pattern.sub(lambda x: str(int(x.group(1)) * value), text)

    return text

# Function to evaluate the numerical expression
def evaluate_expression(expression):
    expression = expression.replace(' ', '+')
    return eval(expression)

# Example input text containing Hindi numerals
input_text = "Mujhe chaualise hazaar char sau teen ruppey chahiye."

# Convert Hindi numerals to numerical equivalents
converted_text = convert_hindi_numerals(input_text)

# Evaluate the expression
result = evaluate_expression(converted_text)

# Print the output text
print("Input:", input_text)
print("Converted Text:", converted_text)
print("Result:", result)


SyntaxError: invalid syntax (<string>, line 1)